# Financial Metrics Calculation (Ground Truth)

Input:
- Clean pandas DataFrame from file_parser

Output:
- Dictionary of financial metrics

Rules:
- Deterministic math only
- No AI
- No risk judgement yet

